In [1]:
import os
import numpy as np
from datetime import timedelta

from astropy.io import fits, ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation

from skyfield.api import wgs84, load, Star
from skyfield.framelib import ecliptic_frame

from lvmdrp.core.constants import EPHEMERIS_PATH
from lvmdrp.main import load_master_config

In [2]:
# read survey tables
lvm = ascii.read('_data-sky/survey.txt')
lvm = lvm[lvm['ra']!=-999.0]

lvm.add_column(np.arange(len(lvm), dtype=int), name="i")
lvm.add_column(np.arctan(lvm["lunation"])*180/np.pi, name="moon_sun_sep")

TARGET_SURVEY = "FULLSKY"
survey = lvm[lvm["target"] == TARGET_SURVEY]
survey


JD,observatory,target,group,tileid,index,ra,dec,pa,airmass,lunation,shadow_height,moon_dist,lst,exptime,totaltime,i,moon_sun_sep
float64,str3,str12,str6,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64
2459948.805059729,LCO,FULLSKY,ALLSKY,1024739,1968,129.43820224719101,-5.056179775280882,0.0,1.1230315971337637,0.9314728552535911,1017.4520707911919,64.10752178656843,9.517918542120606,900.0,990.0000000000001,44,42.96804290613094
2459948.8165180623,LCO,FULLSKY,ALLSKY,1023968,1197,138.13953488372093,15.1685393258427,0.0,1.4136401088976378,0.9314728552535911,1015.5793299592212,63.145910661692916,9.793671461970856,900.0,990.0000000000001,45,42.96804290613094
2459949.8051448795,LCO,FULLSKY,ALLSKY,1024516,1745,141.57303370786516,1.011235955056186,0.0,1.1557399289659196,0.9712185433583314,1003.5608431512785,60.58636348513556,9.585677570321907,900.0,990.0000000000001,65,44.16349263765874
2459950.804167936,LCO,FULLSKY,ALLSKY,1023974,1203,163.2558139534884,15.1685393258427,0.0,1.4880082457030084,0.9934526605258615,1005.5890040932535,61.54968812186149,9.627876558123777,900.0,990.0000000000001,84,44.81181716655173
2459951.5979621634,LCO,FULLSKY,ALLSKY,1025556,2785,133.84615384615384,-29.325842696629206,0.0,1.7091921894514037,0.9978511724445136,1013.7359791645517,60.12595675490734,4.731098092564692,900.0,990.0000000000001,85,44.938374437469456
2459951.8042121595,LCO,FULLSKY,ALLSKY,1025551,2780,87.6923076923077,-29.325842696629206,0.0,1.5513166262968223,0.9978511724445136,1010.4672780352439,60.39029730700139,9.694650649713973,900.0,990.0000000000001,103,44.938374437469456
2459953.8073639222,LCO,FULLSKY,ALLSKY,1025432,2661,97.77777777777777,-25.280898876404503,0.0,1.4136020345692777,0.9541877909433125,1037.1324471653768,60.32120685084244,9.90191970669354,900.0,990.0000000000001,125,43.657055608163915
2459962.8345956104,LCO,FULLSKY,ALLSKY,1024359,1588,133.48314606741573,5.056179775280903,0.0,1.46700213389715,0.1872842849831461,1009.0105925320827,110.86407109550203,11.148658031411468,900.0,990.0000000000001,257,10.607715008386648
2459963.8343955865,LCO,FULLSKY,ALLSKY,1024437,1666,133.48314606741573,3.0337078651685503,0.0,1.4435899104977301,0.10535488124849113,1012.978195147929,123.56441141408772,11.20955413930739,900.0,990.0000000000001,265,6.014203675900861


In [42]:
# define observatory
master_config = load_master_config(fmt="dict")
obs_pars = master_config["LVM_OBSERVATORIES"]["LCO"]

astros = load(EPHEMERIS_PATH)
ts = load.timescale()
sun, earth, moon = astros["sun"], astros["earth"], astros["moon"]
obs_topos = wgs84.latlon(latitude_degrees=obs_pars["lat"], longitude_degrees=obs_pars["lon"], elevation_m=obs_pars["height"])
obs = earth + obs_topos

In [43]:
# define target
idx = 10
ra, dec = survey["ra"][idx]*u.deg, survey["dec"][idx]*u.deg
time = ts.tt_jd(survey["JD"][idx])


obs = obs.at(time)

target = Star(ra_hours=ra.to(u.hourangle), dec_degrees=dec.to(u.deg))

tar = obs.observe(target).apparent()

tar_lat, tar_lon, _ = tar.frame_latlon(ecliptic_frame)

In [47]:
from skyfield import almanac

t_ini, t_fin = time - timedelta(days=2), time + timedelta(days=2)

risings_and_settings, _ = almanac.find_discrete(t_ini, t_fin, almanac.sunrise_sunset(ephemeris=astros, topos=obs_topos))
# assert all(rising == [0, 1])

i = np.digitize(time.tt, bins=risings_and_settings.tt, right=False)
risings_and_settings[i-1].tt <= time.tt < risings_and_settings[i].tt, _[[i-1,i]]

(True, array([0, 1], dtype=int8))

In [48]:
night_thirds = np.linspace(*risings_and_settings[[i-1, i]].tt, 4)
night_thirds

array([2459965.49008799, 2459965.63334396, 2459965.77659992,
       2459965.91985589])

In [49]:
np.digitize(time.tt, bins=night_thirds)

3